In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wagonworks.get_data import get_day_data
from wagonworks.utils import format_date, format_time
import datetime

In [6]:
#Get all the orders of a day
jan_day = get_day_data('2020-01-06')
# jan_day

In [8]:
#select orders with same_day flag
same_day_df = jan_day[jan_day['Same_Day_Flag'] == 'X'][['Del_NumA','SKU_A','Act_Goods_Issue_Qty_SKU','Del_Creat_DateTime']]

In [9]:
#Get orders by hour
same_day_df['Hour'] = same_day_df['Del_Creat_DateTime'].dt.hour
same_day_df

,Del_NumA,SKU_A,Act_Goods_Issue_Qty_SKU,Del_Creat_DateTime,Hour
4078,TEST09340346ZA,ZENE07333051,1,2020-01-06 11:41:14,11
15491,TEST09928446ZA,ZJBFZKT18988,1,2020-01-06 15:11:00,15
15492,TEST09928446ZA,ZJBFZ2510887,6,2020-01-06 15:11:00,15
16102,TEST09928446ZA,Z3M6MIA25425,1,2020-01-06 15:11:00,15
16103,TEST09928446ZA,ZBI93AS40806,3,2020-01-06 15:11:00,15
...,...,...,...,...,...
324640,TEST09027646ZA,ZAC302949082,1,2020-01-06 09:51:04,9
324641,TEST09027646ZA,ZSA300152267,1,2020-01-06 09:51:04,9
324642,TEST09027646ZA,ZJB18BK46482,1,2020-01-06 09:51:04,9
324643,TEST09027646ZA,ZORD30126216,4,2020-01-06 09:51:04,9


In [10]:
orders_time = same_day_df['Hour'].min()
orders_time

6

In [11]:
hour_df = same_day_df[same_day_df['Hour'] == orders_time].index

In [12]:
hour_df

Int64Index([ 44284, 194099, 194100, 194101, 194102, 194105, 194106, 194107,
            194108, 194109, 194110, 194111, 194112, 194113, 194114, 194115,
            194116, 194117, 194118, 194119, 194120, 194121, 194122, 194123,
            194124, 194125, 194126, 194127, 194128, 194152, 194153, 194154,
            194155, 194156, 194159, 194160, 194161, 194162, 194163, 194166,
            194167, 194168, 194169, 194170, 194171, 194172, 194173, 194174,
            194175, 194210, 194211, 206289, 206290, 206291, 206292, 206293,
            206294, 206295, 206296, 206297, 206298, 206299, 206309, 206310,
            206330],
           dtype='int64')

In [1]:
def order_pool_ui(csv):
    
    # read in csv file as a df
    single_df = pd.read_csv(csv, low_memory = False)

    # format date columns
    single_df.loc[:,'Plan_GI_Date'] = format_date(single_df, date_col ='Plan_GI_Date')
    single_df.loc[:,'Act_GI_Date'] = format_date(single_df, date_col ='Act_GI_Date')
    single_df.loc[:,'Del_Creat_Date'] = format_date(single_df, date_col ='Del_Creat_Date')

    # format date columns
    single_df.loc[:,'Del_Creat_Time'] = format_time(single_df, time_col = 'Del_Creat_Time')
    
    # creating a loop variable to drop done orders
    loop_dataframe = single_df.copy()
    orders_list = []
    
    while len(loop_dataframe) > 0:

        #condition to prioritise same day orders
        if len(loop_dataframe[loop_dataframe['Same_Day_Flag'] == 'X']) > 0:
            day_flag = loop_dataframe[loop_dataframe['Same_Day_Flag'] == 'X'][['Del_NumA','SKU_A','Act_Goods_Issue_Qty_SKU','Del_Creat_DateTime', 'Same_Day_Flag']]
        else:
            day_flag = loop_dataframe[loop_dataframe['Same_Day_Flag'] != 'X'][['Del_NumA','SKU_A','Act_Goods_Issue_Qty_SKU','Del_Creat_DateTime', 'Same_Day_Flag']]

        #select orders of the first available hour
        day_flag['Hour'] = day_flag['Del_Creat_DateTime'].dt.hour
        orders_time = day_flag['Hour'].min()
        hour_df = day_flag[day_flag['Hour'] == orders_time]
        hour_df_index = day_flag[day_flag['Hour'] == orders_time].index

        #create list of dataframes to be sent to batch
        loop_dataframe.drop(hour_df_index, axis = 0, inplace = True)
        hour_df.reset_index(drop = True, inplace = True)
        orders_list.append(hour_df)

    return orders_list